In [1]:
import pandas as pd
import ccxt
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor


In [2]:
async def async_fetch_ohlcv(exchange: ccxt.Exchange, symbol: str, timeframe: str, since: int, limit: int):
    ohlcv = exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe, since=since, limit=limit)

    return ohlcv

def fetch_binance_ohlcv(symbole: str, timeframe: str, since: int, until: int) -> pd.DataFrame:
    binance = ccxt.binance()
    since_arr = [i for i in range(since, until, 1000*1000)]

    with ThreadPoolExecutor() as executor:
        all_prices = executor.map(lambda since: binance.fetch_ohlcv(symbol=symbole, timeframe=timeframe, since=since, limit=1000), since_arr)

    all_prices = [price for prices in all_prices for price in prices]

    return all_prices

In [3]:
all_prices = fetch_binance_ohlcv('ETH/USDC', '1s', int(datetime(2024, 1, 1, tzinfo=timezone.utc).timestamp() * 1000), int(datetime(2024, 9, 1, tzinfo=timezone.utc).timestamp() * 1000))

In [18]:
df = pd.DataFrame(all_prices, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df["price"] = (df["open"] + df["close"]) / 2
df["time"] = df["time"]//1000
df = df.set_index('time')
df = df.drop(columns=['open', 'high', 'low', 'close', 'volume'])
df.head()

,price
time,
1704067200,2281.57
1704067201,2281.57
1704067202,2281.57
1704067203,2281.65
1704067204,2281.65


In [19]:
df.to_csv('ETHUSDC-2024-01>08_1s.csv')